In [13]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
import keras

In [14]:
from keras.models import Model

In [2]:
from keras.layers import Input, Conv2D,Deconv2D
from configs import configs
from util import make_model_data_struct

Using TensorFlow backend.


ImportError: No module named configs

In [ ]:
from keras.regularizers import l2

In [16]:
from iclr_supervised import encoder, bbox

In [17]:
encoder_num_filters_list = configs["num_filter_list"]
num_layers = len(encoder_num_filters_list)
inp_shape = configs["tensor_input_shape"]

In [18]:
input_ = Input(inp_shape)

In [ ]:
conv_kwargs =  dict(border_mode="same", init=configs["w_init"], W_regularizer=l2(configs["w_decay"]))

In [36]:
def decoder(code_layer_output):
    # us all filter sizes from encoder except for last, most recent one
    decoder_num_filters_list = encoder_num_filters_list[:-1]
    # use them in reverse
    decoder_num_filters_list.reverse()
    
    # so the last layer gets to shape of input, last layer num_filters == input num channels
    decoder_num_filters_list.append(inp_shape[-1])

    x = code_layer_output
    
    for lay_no in range(num_layers):
        num_filters = decoder_num_filters_list[lay_no]
        o_shape = [None]+ [dim / 2**(num_layers - lay_no -1) for dim in inp_shape[0:2]] + [num_filters]
        #print o_shape
        if lay_no == num_layers - 1:
            name = "reconstruction"
        else:
            name = "deconv_" + str(lay_no)
        x = Deconv2D(num_filters, 5,5,subsample=(2,2), output_shape=o_shape,  name=name, **conv_kwargs)(x)


    return x

In [37]:
code_layer_output = encoder(inp=input_)

In [38]:
reconstruction = decoder(code_layer_output)

[None, 48, 48, 96]
[None, 96, 96, 64]
[None, 192, 192, 32]
[None, 384, 384, 16]
[None, 768, 768, 16]


In [23]:
box_score= bbox(code_layer_output)

In [24]:
def get_model_params():
    return make_model_data_struct(input=input_, output = [box_score, reconstruction])

In [1]:
#! jupyter nbconvert --to script iclr_semisupervised.ipynb

[NbConvertApp] Converting notebook iclr_semisupervised.ipynb to script
[NbConvertApp] Writing 1829 bytes to iclr_semisupervised.py
